To investigate why NN predictions are lower than calculated theoretical values I've decided to eplore what predictions mean predictor does. I've fixed the seed when selecting items to do predictions, so the predictions are made on the same items. Here are NN predictions:

| No label | With label |
| --- | --- |
| <img src="1/order_plot_nn_no_label.png"/> | <img src="1/order_plot_nn_with_label.png"/> |

And the mean predictor:

<img src="1/order_plot_mean.png" style="width: 700px;"/>

Predictions made by mean predictor are also seem to be biased to be lower than calculated theoretical values. Let's check that the problem is not in generated trace. In the trace the ratio of items from first and second population should be 1:1.

Number of requests in trace:
<img src="2/all.png"/>

First population: 4997831 <br/>
Second population: 5002169 <br/>
There are even slightly more requests from second population, so the problem is not in trace.

While closely inspecting predictions made by mean predictor on second population I've observed that the mean of predictions is close to caclulated theoretical value: $ 9.99487524808 * 10^{-5} $. <br/>
But the median is $5.85074972881 * 10^{-5}$. <br/>
Low median of the predictions and preference to low popularity items after $ -log(frac + 10^{-5}) $ transformation is the possible explanation of behaviour of the NNs.

For comparison, mean and median of predictions made by NN.<br/> 
Case with label: <br/>
Mean: $ 5.41192173486 * 10^{-5} $ <br/>
Median: $ 5.4169656957 * 10^{-5} $ <br/>
Case without label: <br/>
Mean: $ 4.81817910687 * 10^{-5} $ <br/>
Median: $ 4.69565798293 * 10^{-5} $

Now let's address the problem of validation error being lower than training error. <br/>
First of all, I've hound a seed for the random generator using which the validation error is smaller than training error after first few iterations. Let's confirm that it doesn't become larger eventually.

<img src="3_err_plot.png" style="width: 700px;"/>

The validation error stays lower even after a large number of iterations. But in case of using average predictor (which is the best linear predictor) the error on validation set is also smaller, and the difference between these two errors is close to the one present in NN error evaluation. From this it is possible to conclude that the initial training/validation split influences how the train and validation set errors will compare. Let's establish an experiment - to compare how the difference between training and validation errors compares when using NN and average predictor and different train/validation split.

<img src="4_split_error_plot.png" style="width: 700px;"/>

Testing 100 different splits the trend is apparent - the difference is always close when using NN with Leaky ReLU activation and average predictor. <br/>
Maybe Leaky ReLU activation does not introduce enough nonlinearity to allow NN overfit the training set and produce lower error on the training set always. Let's try to use SoftPlus, which is $ ln(1 + e^x) $, as activation.
<img src="5_softplus.png"/>

|Error curves|Predictions|
| --- | --- |
|<img src="6/err_plot.png"/>|<img src="6/order_plot.png"/>|

SoftMax activation didn't solve the problem and even showed higher error than ReLU.

Following Giovanni's advice, it is possible that the used constant $ 10^{-5} $ is too high. Let's chech what results will $10^{-8}$ show.

|Error curves|Predictions|
| --- | --- |
|No label <img src="7/1_err_plot.png"/>|No label<img src="7/1_order_plot.png"/>|
|With label <img src="7/2_err_plot.png"/>|With label<img src="7/2_order_plot.png"/>|